This notebook was prepared by [Author](https://github.com/). Source and license info is on [GitHub](https://github.com/donnemartin/interactive-coding-challenges).

# Solution Notebook

## Problem: 

### Implement Linear Regression model, which has a predict function which makes prediction on input data using **Tensor Flow** library

* [Constraints](#Constraints)
* [Test Cases](#Test-Cases)
* [Algorithm](#Algorithm)
* [Code](#Code)
* [Unit Test](#Unit-Test)
* [Solution Notebook](#Solution-Notebook)

## Constraints

* Can we assume all predictor variable are numerical?
    * Yes
* Can we assume no missing value in data?
    * Yes
* Can we assume no outlier in data?
    * Yes

## Algorithm

Implement Linear Regression using Tensorflow library

## Data

Get data from: 

    from sklearn import datasets
    data = datasets.load_diabetes()

## Code

In [2]:
from sklearn import datasets
import tensorflow as tf
data = datasets.load_diabetes()

class linear_regression():
    def train(X_train, Y_train):
        # TODO: Implement me
        pass
    
    def predict(X_test):
        # TODO: Implement me
        pass

## Unit Test

In [5]:
%%writefile test_tf_linear_regression.py
from nose.tools import assert_equal


class TestFoo(object):

    def test_foo(self):
        assert_equal(foo(None), None)
        assert_equal(foo(0), 0)
        assert_equal(foo('bar'), 'bar')
        print('Success: test_foo')


def main():
    test = TestFoo()
    test.test_foo()


if __name__ == '__main__':
    main()

Overwriting test_tf_linear_regression.py


In [3]:
%run -i test_tf_linear_regression.py

Success: test_foo
